### 已知前向传播过程：
(1）更新遗忘门输出：
\begin{eqnarray}
f^{(t)} = \sigma(W_fh^{(t-1)} + U_fx^{(t)} + b_f)
\end{eqnarray}
(2）更新输入门两部分输出：
\begin{eqnarray}
&&i^{(t)} = \sigma(W_ih^{(t-1)} + U_ix^{(t)} + b_i)\\
\\
&&a^{(t)} = tanh(W_ah^{(t-1)} + U_ax^{(t)} + b_a)
\end{eqnarray}

(3）更新细胞状态：
\begin{eqnarray}
C^{(t)} = C^{(t-1)} \odot f^{(t)} + i^{(t)} \odot a^{(t)}
\end{eqnarray}
(4）更新输出门输出：
\begin{eqnarray}
&&o^{(t)} = \sigma(W_oh^{(t-1)} + U_ox^{(t)} + b_o)\\
\\
&&h^{(t)} = o^{(t)} \odot tanh(C^{(t)})
\end{eqnarray}
(5）更新当前序列索引预测输出：
\begin{eqnarray}
\hat{y}^{(t)} = \sigma(Vh^{(t)} + c)
\end{eqnarray}

### 相比RNN，LSTM中有两个隐藏状态$h^{(t)}$和$C^{(t)}$，于是定义两个误差$\delta$：
\begin{eqnarray}
&&\delta_h^{(t)} = \frac{\partial L}{\partial h^{(t)}}\\
&&\delta_C^{(t)} = \frac{\partial L}{\partial C^{(t)}}
\end{eqnarray}

### 定义损失函数L(t)：
\begin{eqnarray}
L(t) = \begin{cases} l(t) + L(t+1) & \text{if} \, t < \tau \\ l(t) & \text{if} \, t = \tau\end{cases}
\end{eqnarray}
### 首先，计算序列索引位置τ的$\delta_h^{(\tau)}$和 $\delta_C^{(\tau)}$为:
\begin{eqnarray}
&&\delta_h^{(\tau)} =\frac{\partial L^{(\tau)}}{\partial O^{(\tau)}} \frac{\partial O^{(\tau)}}{\partial h^{(\tau)}} = V^T(\hat{y}^{(\tau)} - y^{(\tau)})\\
\\
&&\delta_C^{(\tau)} =\frac{\partial L^{(\tau)}}{\partial h^{(\tau)}} \frac{\partial h^{(\tau)}}{\partial C^{(\tau)}} = \delta_h^{(\tau)} \odot  o^{(\tau)} \odot (1 - tanh^2(C^{(\tau)}))
\end{eqnarray}
### 接着，由$\delta_C^{(t+1)},\delta_h^{(t+1)}$反向推导$\delta_h^{(t)}, \delta_C^{(t)}$：
$\delta_h^{(t)}$由本层t时刻的输出梯度误差和t+1时刻的误差两部分决定，即：
\begin{eqnarray}
\delta_h^{(t)} =\frac{\partial L}{\partial h^{(t)}}  =\frac{\partial l(t)}{\partial h^{(t)}} + \frac{\partial L(t+1)}{\partial h^{(t+1)}}  \frac{\partial h^{(t+1)}}{\partial h^{(t)}} = V^T(\hat{y}^{(t)} - y^{(t)}) + \delta_h^{(t+1)} \frac{\partial h^{(t+1)}}{\partial h^{(t)}}
\end{eqnarray}
现在的重点是求解$\frac{\partial h^{(t+1)}}{\partial h^{(t)}}$.
由于：
\begin{eqnarray}
&&h^{(t)} = o^{(t)} \odot tanh(C^{(t)})\\
\end{eqnarray}
在第一项$o^{(t)}$中，包含一个h的递推关系。   
第二项$tanh(C^{(t)})$中，由$C^{(t)} $表达式：
\begin{eqnarray}
C^{(t)} = C^{(t-1)} \odot f^{(t)} + i^{(t)} \odot a^{(t)}
\end{eqnarray}
$f^{(t)}$包含一个h的递推关系，$i^{(t)}$ 和$a^{(t)}$都包含h的递推关系。因此，$\frac{\partial h^{(t+1)}}{\partial h^{(t)}}$由四部分组成。

设$\Delta C = o^{(t+1)} \odot [1-tanh^2(C^{(t+1)})]$，可得： 
<br>
<br>
\begin{eqnarray}
\frac{\partial h^{(t+1)}}{\partial h^{(t)}} = W_o^T [o^{(t+1)} \odot (1-o^{(t+1)}) \odot tanh(C^{(t+1)})] +  W_f^T [\Delta C  \odot f^{(t+1)} \odot (1-f^{(t+1)}) \odot C^{(t)}] + W_a^T \{ \Delta C  \odot i^{(t+1)} \odot [1-(a^{(t+1)})^2] \}  + W_i^T [\Delta C  \odot a^{(t+1)} \odot  i^{(t+1)}  \odot (1-i^{(t+1)})]
\end{eqnarray}

$\delta_C^{(t)}$由$\delta_C^{(t+1)}$和$h^{(t)}$组成，即：
\begin{eqnarray}
\delta_C^{(t)} =\frac{\partial L}{\partial C^{(t+1)}} \frac{\partial  C^{(t+1)}}{\partial C^{(t)}} + \frac{\partial L}{\partial h^{(t)}}\frac{\partial h^{(t)}}{\partial C^{(t)}} = \delta_C^{(t+1)}\odot f^{(t+1)} + \delta_h^{(t)} \odot  o^{(t)} \odot (1 - tanh^2(C^{(t)}))
\end{eqnarray}
### 最后，有了$\delta_h^{(t)}$和$\delta_C^{(t)}$便可求得$U_f, b_f, W_a, U_a, b_a, W_i, U_i, b_i, W_o, U_o, b_o，V, c$：
\begin{eqnarray}
&&\frac{\partial L}{\partial W_f} =  \sum\limits_{t=1}^{\tau}\frac{\partial L}{\partial C^{(t)}} \frac{\partial C^{(t)}}{\partial f^{(t)}} \frac{\partial f^{(t)}}{\partial W_f} =\sum\limits_{t=1}^{\tau} [\delta_C^{(t)} \odot C^{(t-1)} \odot f^{(t)}\odot(1-f^{(t)})] (h^{(t-1)})^T\\
&&......
\end{eqnarray}
